In [6]:
import numpy as np

file_path = "bible.txt"

with open(file_path, "r") as f:
    text = f.read()

In [10]:

vocabulary = {x: i for i, x in enumerate(np.unique(list(text)))}
char_tokens = [vocabulary[char] for char in text]
print(char_tokens[:50])

[42, 55, 52, 1, 28, 56, 65, 66, 67, 1, 24, 62, 62, 58, 1, 62, 53, 1, 35, 62, 66, 52, 66, 20, 1, 1, 25, 48, 59, 59, 52, 51, 1, 29, 52, 61, 52, 66, 56, 66, 0, 0, 0, 11, 20, 11, 1, 31, 61, 1]
